In [ ]:
import random
import math
import matplotlib.pyplot as plt

# PARÂMETROS VARIÁVEIS (Ajustar aqui para testes)
TAMANHO_POPULACAO = 4      # População inicial = 4; Pode aumentar para 30
NUM_GERACOES = 5           # N° inicial de gerações = 5; Pode aumentar para 20
TAXA_CROSSOVER = 0.70      # 70%
TAXA_MUTACAO = 0.01        # 1%
BITS_POR_INDIVIDUO = 10    # Quanto maior, mais precisão decimal
INTERVALO = [-10, 10]      # X varia de -10 a +10
TORNEIO_SIZE = 2           # Tamanho do torneio

In [ ]:
def binario_para_real(cromossomo):
    """
    Converte um vetor binário para um número real dentro do intervalo [-10, 10].
    """
    # Converte lista de bits para inteiro
    inteiro = int("".join(map(str, cromossomo)), 2)
    max_inteiro = 2**BITS_POR_INDIVIDUO - 1

    # Mapeia o inteiro para o intervalo desejado
    min_x, max_x = INTERVALO
    valor_real = min_x + (inteiro / max_inteiro) * (max_x - min_x)
    return valor_real

def funcao_objetivo(x):
    """
    Função f(x) = x^2 - 3x + 4
    """
    return x**2 - 3*x + 4

def calcular_fitness(cromossomo):
    """
    Calcula o fitness de um indivíduo (quanto maior, melhor).
    """
    x = binario_para_real(cromossomo)
    return funcao_objetivo(x)

def criar_individuo():
    """Gera um indivíduo aleatório (vetor de bits)."""
    return [random.randint(0, 1) for _ in range(BITS_POR_INDIVIDUO)]

def criar_populacao(tamanho):
    """Cria a população inicial."""
    return [criar_individuo() for _ in range(tamanho)]

In [ ]:
def selecao_torneio(populacao):
    """
    Seleciona um indivíduo por torneio.
    Escolhe 'k' indivíduos aleatórios e retorna o melhor.
    """
    competidores = random.sample(populacao, TORNEIO_SIZE)
    melhor = max(competidores, key=calcular_fitness)
    return melhor

def crossover(pai1, pai2):
    """
    Aplica crossover de um ponto com base na TAXA_CROSSOVER.
    """
    if random.random() < TAXA_CROSSOVER:
        ponto_corte = random.randint(1, BITS_POR_INDIVIDUO - 1)
        filho1 = pai1[:ponto_corte] + pai2[ponto_corte:]
        filho2 = pai2[:ponto_corte] + pai1[ponto_corte:]
        return filho1, filho2
    else:
        return pai1[:], pai2[:] # Retorna cópias se não houver crossover

def mutacao(individuo):
    """
    Aplica mutação bit-flip com base na TAXA_MUTACAO.
    """
    novo_ind = []
    for bit in individuo:
        if random.random() < TAXA_MUTACAO:
            novo_ind.append(1 if bit == 0 else 0) # Inverte o bit
        else:
            novo_ind.append(bit)
    return novo_ind

In [ ]:
# @title
# --- EXECUÇÃO DO ALGORITMO ---

# 1. População Inicial
populacao = criar_populacao(TAMANHO_POPULACAO)
historico_fitness_melhor = []
historico_fitness_media = [] # NOVA LISTA

print(f"--- Iniciando AG com {TAMANHO_POPULACAO} indivíduos por {NUM_GERACOES} gerações ---\n")

for geracao in range(NUM_GERACOES):
    nova_populacao = []

    # Avaliação de TODOS os indivíduos
    valores_fitness = [calcular_fitness(ind) for ind in populacao]

    # Estatísticas da Geração
    melhor_fitness = max(valores_fitness)
    media_fitness = sum(valores_fitness) / len(populacao) # CÁLCULO DA MÉDIA

    # Identifica o melhor indivíduo para mostrar o X
    indice_melhor = valores_fitness.index(melhor_fitness)
    melhor_ind = populacao[indice_melhor]
    melhor_x = binario_para_real(melhor_ind)

    # Salva no histórico para o gráfico
    historico_fitness_melhor.append(melhor_fitness)
    historico_fitness_media.append(media_fitness)

    # Print mais completo
    print(f"Geração {geracao+1}: Melhor X = {melhor_x:.4f} | Melhor Fit = {melhor_fitness:.4f} | Média Fit = {media_fitness:.4f}")

    # Loop para criar nova geração (Reprodução)
    while len(nova_populacao) < TAMANHO_POPULACAO:
        pai1 = selecao_torneio(populacao)
        pai2 = selecao_torneio(populacao)

        f1, f2 = crossover(pai1, pai2)

        f1 = mutacao(f1)
        f2 = mutacao(f2)

        nova_populacao.append(f1)
        if len(nova_populacao) < TAMANHO_POPULACAO:
            nova_populacao.append(f2)

    populacao = nova_populacao

# --- RESULTADO FINAL ---
valores_fitness = [calcular_fitness(ind) for ind in populacao]
melhor_geral_fit = max(valores_fitness)
melhor_geral_ind = populacao[valores_fitness.index(melhor_geral_fit)]
melhor_geral_x = binario_para_real(melhor_geral_ind)

print("\n" + "="*40)
print("RESULTADO FINAL")
print(f"Melhor X encontrado: {melhor_geral_x:.5f}")
print(f"Valor Máximo da Função: {melhor_geral_fit:.5f}")
print("="*40)

# Análise Matemática Rápida
# A parábola abre para cima. No intervalo [-10, 10]:
# f(-10) = 100 + 30 + 4 = 134
# f(10) = 100 - 30 + 4 = 74
# O algoritmo deve convergir para perto de -10.

In [ ]:
from matplotlib.ticker import MaxNLocator

plt.figure(figsize=(10, 6))

# Linhas do Algoritmo
plt.plot(historico_fitness_melhor, label='Melhor da Geração', marker='o', color='blue')
plt.plot(historico_fitness_media, label='Média da População', linestyle='--', color='orange')

# A Linha de Chegada (Máximo Teórico)
plt.axhline(y=134, color='red', linestyle='-', linewidth=2, label='Máximo Teórico (134)')

plt.title('Distância até a Solução Ótima (Target)')
plt.xlabel('Geração')
plt.ylabel('Valor de Fitness')
plt.legend()
plt.grid(True, alpha=0.3)
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

plt.show()